In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
movs = [file for file in os.listdir('./') if '.mov' in file]
rgbs = sorted([file for file in movs if 'RGB' in file])
thrs = sorted([file for file in movs if 'THR' in file])
rgb2thr = {}
for rgb,thr in zip(rgbs,thrs):
    rgb2thr[rgb] = thr
rgb2thr

{'jungjaRGB10_0204DJI_0055.mov': 'jungjaTHR10_0204DJI_0056.mov',
 'jungjaRGB2_190-2.mov': 'jungjaTHR2_190-2.mov',
 'jungjaRGB3_93-1.mov': 'jungjaTHR3_93-1.mov',
 'jungjaRGB4_102(2)_0205DJI_0031.mov': 'jungjaTHR4_102(2)_0205DJI_0032.mov',
 'jungjaRGB5_102_0205DJI_0021.mov': 'jungjaTHR5_102_0205DJI_0022.mov',
 'jungjaRGB6_122_0205DJI_0057.mov': 'jungjaTHR6_122_0205DJI_0058.mov',
 'jungjaRGB7_197_0206DJI_0013.mov': 'jungjaTHR7_197_0206DJI_0014.mov',
 'jungjaRGB8_198_0204DJI_0088.mov': 'jungjaTHR8_198_0204DJI_0087.mov',
 'jungjaRGB9_210-1DJI_0071.mov': 'jungjaTHR9_210-1DJI_0072.mov',
 'soonaeRGB1_68-1.mov': 'soonaeTHR1_68-1.mov'}

In [6]:
cap = cv2.VideoCapture(rgb)
capTH = cv2.VideoCapture(thr)

In [7]:
RGBcol = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
RGBrow = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
THRcol = int(capTH.get(cv2.CAP_PROP_FRAME_WIDTH))
THRrow = int(capTH.get(cv2.CAP_PROP_FRAME_HEIGHT))
print('RGB ',(RGBcol,RGBrow),' THR',(THRcol,THRrow))

RGB  (3840, 2160)  THR (640, 512)


In [8]:
RGBfps = cap.get(cv2.CAP_PROP_FPS)
THRfps = capTH.get(cv2.CAP_PROP_FPS)
print(RGBfps,THRfps)

29.72 29.15


In [9]:
cap.release()
capTH.release()

In [11]:
for rgb,thr in rgb2thr.items():
    cap = cv2.VideoCapture(rgb)
    capTH = cv2.VideoCapture(thr)
    print(rgb,'>>> total frame count:',cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(thr,'>>> total frame count:',capTH.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    capTH.release()

jungjaRGB10_0204DJI_0055.mov >>> total frame count: 449.0
jungjaTHR10_0204DJI_0056.mov >>> total frame count: 446.0
jungjaRGB2_190-2.mov >>> total frame count: 3330.0
jungjaTHR2_190-2.mov >>> total frame count: 3257.0
jungjaRGB3_93-1.mov >>> total frame count: 2362.0
jungjaTHR3_93-1.mov >>> total frame count: 2303.0
jungjaRGB4_102(2)_0205DJI_0031.mov >>> total frame count: 3557.0
jungjaTHR4_102(2)_0205DJI_0032.mov >>> total frame count: 3485.0
jungjaRGB5_102_0205DJI_0021.mov >>> total frame count: 2715.0
jungjaTHR5_102_0205DJI_0022.mov >>> total frame count: 2675.0
jungjaRGB6_122_0205DJI_0057.mov >>> total frame count: 6174.0
jungjaTHR6_122_0205DJI_0058.mov >>> total frame count: 6037.0
jungjaRGB7_197_0206DJI_0013.mov >>> total frame count: 8589.0
jungjaTHR7_197_0206DJI_0014.mov >>> total frame count: 8320.0
jungjaRGB8_198_0204DJI_0088.mov >>> total frame count: 12143.0
jungjaTHR8_198_0204DJI_0087.mov >>> total frame count: 11802.0
jungjaRGB9_210-1DJI_0071.mov >>> total frame count: 33

fps 비율에 따라 프레임 추출 하는 코드

In [3]:
#filenames = [filename for filename in os.listdir('./') if 'RGB' in filename and '.mov' in filename]

for filename,THR_filename in rgb2thr.items():
    if not os.path.exists(filename.split('.')[0]) and 'RGB':
        os.mkdir(filename.split('.')[0])

    dirname = filename.split('.')[0]
    print(filename,THR_filename)

    cap = cv2.VideoCapture(filename)
    capTH = cv2.VideoCapture(THR_filename)

    rowDis,colDis = 455,555
    # RGB 영상의 중간 픽셀을 기준으로 위아래로 455 픽셀씩, 좌우로 555픽셀씩 잘라어 사용

    idx = 0
    THR_WIN_NAME = 'THERMAL'

    RGBcol = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    RGBrow = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    THRcol = int(capTH.get(cv2.CAP_PROP_FRAME_WIDTH))
    THRrow = int(capTH.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ratio = cap.get(cv2.CAP_PROP_FRAME_COUNT) / capTH.get(cv2.CAP_PROP_FRAME_COUNT)

    #RGBrow,RGBcol = frame.shape[0],frame.shape[1]
    #THRrow,THRcol = frameTH.shape[0],frameTH.shape[1]
    leftUpRow = RGBrow//2 - THRrow//2
    leftUpCol = RGBcol//2 - THRcol//2
    rightDownRow = leftUpRow + THRrow
    rightDownCol = leftUpCol + THRcol
    print((leftUpRow,leftUpCol),(rightDownRow,rightDownCol))
    fileid = 0

    while(cap.isOpened() and capTH.isOpened()):
        retTH,frameTH = capTH.read()

        if idx != 0 and (round(idx*ratio) - round((idx-1)*ratio)) > 1:
            cap.set(cv2.CAP_PROP_POS_FRAMES,round(idx*ratio))
        ret, frame = cap.read()
        if not retTH or not ret: break

        frame = cv2.resize(frame[leftUpRow-rowDis:rightDownRow+rowDis,leftUpCol-colDis:rightDownCol+colDis],dsize=(frameTH.shape[1],frameTH.shape[0]))
        label_img = np.zeros((frameTH.shape[0],frameTH.shape[1],3), np.uint8)

        #frame = cv2.resize(frame[leftUpRow-rowDis:rightDownRow+rowDis,leftUpCol-colDis:rightDownCol+colDis],dsize=(frameTH.shape[1],frameTH.shape[0]))

        cv2.imshow('frame',frame)
        cv2.imshow(THR_WIN_NAME,frameTH)

        
        if idx % 10 == 0 : 
            #print(cap.get(cv2.CAP_PROP_POS_FRAMES),'/',cap.get(cv2.CAP_PROP_FRAME_COUNT), '--',capTH.get(cv2.CAP_PROP_POS_FRAMES),'/',capTH.get(cv2.CAP_PROP_FRAME_COUNT))
            print(cap.get(cv2.CAP_PROP_POS_FRAMES),capTH.get(cv2.CAP_PROP_POS_FRAMES))
            cv2.imwrite(dirname+'/RGB%d.png'%(fileid),frame)
            cv2.imwrite(dirname+'/THR%d.png'%(fileid),frameTH)
            fileid+=1

        idx+=1

        if cv2.waitKey(1) & 0xFF == ord('q'):
            #print('RGB original shape:',(leftUpRow-dis:rightDownRow+dis,leftUpCol-dis:rightDownCol+dis))
            print(frame.shape)
            print(frameTH.shape)
            break
    cap.release()
    capTH.release()
    cv2.destroyAllWindows()


 5311.0
5475.0 5321.0
5485.0 5331.0
5495.0 5341.0
5506.0 5351.0
5516.0 5361.0
5526.0 5371.0
5536.0 5381.0
5547.0 5391.0
5557.0 5401.0
5567.0 5411.0
5578.0 5421.0
5588.0 5431.0
5598.0 5441.0
5608.0 5451.0
5619.0 5461.0
5629.0 5471.0
5639.0 5481.0
5650.0 5491.0
5660.0 5501.0
5670.0 5511.0
5680.0 5521.0
5691.0 5531.0
5701.0 5541.0
5711.0 5551.0
5722.0 5561.0
5732.0 5571.0
5742.0 5581.0
5753.0 5591.0
5763.0 5601.0
5773.0 5611.0
5783.0 5621.0
5794.0 5631.0
5804.0 5641.0
5814.0 5651.0
5825.0 5661.0
5835.0 5671.0
5845.0 5681.0
5855.0 5691.0
5866.0 5701.0
5876.0 5711.0
5886.0 5721.0
5897.0 5731.0
5907.0 5741.0
5917.0 5751.0
5927.0 5761.0
5938.0 5771.0
5948.0 5781.0
5958.0 5791.0
5969.0 5801.0
5979.0 5811.0
5989.0 5821.0
5999.0 5831.0
6010.0 5841.0
6020.0 5851.0
6030.0 5861.0
6041.0 5871.0
6051.0 5881.0
6061.0 5891.0
6071.0 5901.0
6082.0 5911.0
6092.0 5921.0
6102.0 5931.0
6113.0 5941.0
6123.0 5951.0
6133.0 5961.0
6143.0 5971.0
6154.0 5981.0
6164.0 5991.0
6174.0 6001.0
6185.0 6011.0
6195.0 6021.

비율 확인

In [8]:
for filename,THR_filename in rgb2thr.items():
    if not os.path.exists(filename.split('.')[0]) and 'RGB':
        os.mkdir(filename.split('.')[0])

    dirname = filename.split('.')[0]
    print(filename,THR_filename)

    cap = cv2.VideoCapture(filename)
    capTH = cv2.VideoCapture(THR_filename)
    RGBcol = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    RGBrow = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    THRcol = int(capTH.get(cv2.CAP_PROP_FRAME_WIDTH))
    THRrow = int(capTH.get(cv2.CAP_PROP_FRAME_HEIGHT))

    ratio = cap.get(cv2.CAP_PROP_FRAME_COUNT) / capTH.get(cv2.CAP_PROP_FRAME_COUNT)
    print(ratio)

    cap.release()
    capTH.release()


jungjaRGB10_0204DJI_0055.mov jungjaTHR10_0204DJI_0056.mov
1.006726457399103
jungjaRGB2_190-2.mov jungjaTHR2_190-2.mov
1.0224132637396377
jungjaRGB3_93-1.mov jungjaTHR3_93-1.mov
1.0256187581415546
jungjaRGB4_102(2)_0205DJI_0031.mov jungjaTHR4_102(2)_0205DJI_0032.mov
1.0206599713055955
jungjaRGB5_102_0205DJI_0021.mov jungjaTHR5_102_0205DJI_0022.mov
1.0149532710280373
jungjaRGB6_122_0205DJI_0057.mov jungjaTHR6_122_0205DJI_0058.mov
1.0226933907569986
jungjaRGB7_197_0206DJI_0013.mov jungjaTHR7_197_0206DJI_0014.mov
1.0323317307692308
jungjaRGB8_198_0204DJI_0088.mov jungjaTHR8_198_0204DJI_0087.mov
1.0288934078969667
jungjaRGB9_210-1DJI_0071.mov jungjaTHR9_210-1DJI_0072.mov
1.0231354642313546
soonaeRGB1_68-1.mov soonaeTHR1_68-1.mov
1.0237073831564687


In [9]:
29.73/28.81

1.0319333564734468

In [13]:
cap = cv2.VideoCapture('jungjaRGB7_197_0206DJI_0013.mov')
capTH = cv2.VideoCapture('jungjaTHR7_197_0206DJI_0014.mov')
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(capTH.get(cv2.CAP_PROP_FRAME_COUNT))
print(cap.get(cv2.CAP_PROP_FRAME_COUNT)/capTH.get(cv2.CAP_PROP_FRAME_COUNT))
print(cap.get(cv2.CAP_PROP_FPS))
print(capTH.get(cv2.CAP_PROP_FPS))
print(cap.get(cv2.CAP_PROP_FPS)/capTH.get(cv2.CAP_PROP_FPS))
cap.release()
capTH.release()


8589.0
8320.0
1.0323317307692308
29.73
28.81
1.0319333564734468
